# Prediction of Premier League Results

In [75]:
import pandas as pd 
seasons = ['1718','1819','1920','2021']
full_teamdata = []
for season in seasons:
    table = pd.read_csv(season + 'table.csv')
    forteam = pd.read_csv(season + 'for.csv')
    against = pd.read_csv(season + 'against.csv')
    shot = pd.read_csv(season + 'shot.csv')
    season_full = pd.concat([table,forteam,against,shot],axis=1)
    full_teamdata.append(season_full)
full_teamtable = pd.concat(full_teamdata,ignore_index=True)

In [110]:
matches = pd.read_csv('matches.csv')
promoted1819 = ["Wolverhampton Wanderers","Cardiff City","Fulham"]
promoted1920 = ["Norwich City","Sheffield United","Aston Villa"]
promoted2021 = ["Leeds United", "West Bromwich Albion","Fulham"]
promoted2122 = ["Norwich City","Watford","Brentford"]
matches = matches[matches["league"].isin(["2018/2019 English Premier League", "2019-20 English Premier League","2020-21 English Premier League","2021-22 English Premier League"])]
matches = matches[((matches["league"] == "2018/2019 English Premier League") & (~matches["home"].isin(promoted1819)) & (~matches["away"].isin(promoted1819))) | ((matches["league"] == "2019-20 English Premier League") & (~matches["home"].isin(promoted1920)) & (~matches["away"].isin(promoted1920))) | ((matches["league"] == "2020-21 English Premier League") & (~matches["home"].isin(promoted2021)) & (~matches["away"].isin(promoted2021))) | ((matches["league"] == "2021-22 English Premier League") & (~matches["home"].isin(promoted2122)) & (~matches["away"].isin(promoted2122)))]

                         home                    away                   date  \
6462        Manchester United          Leicester City      Friday, August 10   
6463         Newcastle United       Tottenham Hotspur    Saturday, August 11   
6466        Huddersfield Town                 Chelsea    Saturday, August 11   
6467                  Watford  Brighton & Hove Albion    Saturday, August 11   
6469                Liverpool         West Ham United      Sunday, August 12   
...                       ...                     ...                    ...   
7724                  Burnley          Crystal Palace  Saturday, November 20   
7728  Wolverhampton Wanderers         West Ham United  Saturday, November 20   
7729                Liverpool                 Arsenal  Saturday, November 20   
7730          Manchester City                 Everton    Sunday, November 21   
7731        Tottenham Hotspur            Leeds United    Sunday, November 21   

      year time (utc) attendance       